In [1]:
import pandas as pd
import numpy as np

### Reading dataframes for each protein into python 
Renaming columns to be easily accessable, normalizing the excitation and emission spectra, taking sum of column 

In [12]:
df_mEmerald = pd.read_csv('mEmerald_fpbase_spectra.csv')
df_mEmerald.fillna(0, inplace = True)
df_mTagBFP2 = pd.read_csv('mTagBFP2_fpbase_spectra.csv')
df_mTagBFP2.fillna(0, inplace = True)
df_mCherry = pd.read_csv('mCherry_fpbase_spectra.csv')
df_mCherry.fillna(0, inplace = True)
df_mNeptune2p5 = pd.read_csv('mNeptune2p5_fbpase_spectra.csv')
df_mNeptune2p5.fillna(0, inplace = True)

mEmerald_columns = df_mEmerald.rename(columns ={'mEmerald ex': 'excitation', 'mEmerald em': 'emission'})
mTagBFP2_columns = df_mTagBFP2.rename(columns ={'mTagBFP2 ex': 'excitation', 'mTagBFP2 em': 'emission'})
mCherry_columns = df_mCherry.rename(columns ={'mCherry ex': 'excitation', 'mCherry em': 'emission'})
mNeptune2p5_columns = df_mNeptune2p5.rename(columns ={'mNeptune2.5 ex': 'excitation', 'mNeptune2.5 em': 'emission'})

spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]
for j in range(0,4):
    excitation_column = spectra_list[j]['excitation']
    area = np.trapz(excitation_column, dx = 1)
    normalized = excitation_column/area
    excitation_column = normalized
    spectra_list[j]['excitation'] = normalized
    new_area = np.trapz(excitation_column, dx = 1)
    column = spectra_list[j]['emission']
    area_emission = np.trapz(column, dx = 1)
    normalized_em = column/area_emission
    column = normalized_em
    spectra_list[j]['emission'] = normalized_em
    new_area_em = np.trapz(column, dx = 1)

### Function to solve emission bin area using i and j 
Corresponding lambda_min and lambda_max to the min and max wavelengths in channel i in a list of the 200 total bins 

In [22]:
spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]

def calc_emission_bin(i, j):
    bins_list = [*range(400, 900, 10)]
    total_bins = 4*bins_list 
    df_spectra = spectra_list[j]
    lambda_min = total_bins[i]
    lambda_max = total_bins[i+1]
    bin_i = range(lambda_min, lambda_max)
    df_sliced_data = df_spectra.loc[df_spectra['wavelength'].isin(bin_i)]
    # print(df_sliced_data)
    sliced_data_emmision = df_sliced_data['emission']
    emission_bin = np.trapz(sliced_data_emmision)
    print('Emission bin sum for i = ', i, 'and j = ', j, 'is', emission_bin)
    return emission_bin

### Function to retrieve excitation value using j 
Excitation value corresponding to excitation wavelenghts for each protein: 405nm, 488nm, 561nm, 637nm. Found these values on csv files

In [5]:
mEmerald_excitation = [0.1732, 0.9982, 0.0043, 0.0027]
mCherry_excitation = [0.0198, 0.0762, 0.6394, 0.0067]
mTag_excitation = [0.9588, 0, 0, 0]
mNeptune_excitation = [0.0279, 0.0717, 0.5819, 0.1519]

In [18]:
excitation_list = [mEmerald_excitation, mTag_excitation, mCherry_excitation, mNeptune_excitation]
def calc_excitation(i, j): 
     excitation_protein = excitation_list[j]
     if 0 <= i < 50: 
          excitation = excitation_protein[0]
     elif 50 <= i < 100: 
          excitation = excitation_protein[1]
     elif 100 <= i < 150:
          excitation = excitation_protein[2]
     elif 150 <= i < 200:
          excitation = excitation_protein[3]
     else:
          print('Invalid i value')
     print('Excitation value for i = ', i, 'and j = ', j, 'is', excitation)
     return excitation

### Function to index over brightness values using a given j 
Found brightness values on slack 

In [16]:
def calc_brightness(j):
    brightness_list = [39.1, 32.38, 15.85, 22.8]
    brightness = brightness_list[j]
    print('Brightness value for j = ', j, 'is', brightness)
    return brightness

### Function to solve for matrix A just inputting i and j value 
All above functions called in this function to reduce reduncancy in solving for the matrix value

In [23]:
def calc_a(i, j):
    k = 1 
    brightness_value = calc_brightness(j)
    excitation_value = calc_excitation(i, j)
    emission_bin_area = calc_emission_bin(i, j)
    a = k * brightness_value * excitation_value * emission_bin_area
    print('Matrix A for image channel i = ', i, ' and spectrum j = ', j, 'is ', a)
    return a

calc_a(25, 2)

Brightness value for j =  2 is 15.85
Excitation value for i =  25 and j =  2 is 0.0198
Emission bin sum for i =  25 and j =  2 is 0.05754373337904908
Matrix A for image channel i =  25  and spectrum j =  2 is  0.018058949846346973


0.018058949846346973